In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode

In [2]:
import numpy as np
from shapely.geometry import Point, Polygon

In [75]:
import geopandas as gpd

### Seleccionar las bd que conformaran nuestra bd de presencia institucional de Sociedad Civil


-Directorio OIM


In [32]:
directorio_full = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/institucionesmigracionfull.csv')

In [33]:
directorio_full = directorio_full.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

# Resetear el índice y descartar el índice anterior
directorio_full.reset_index(drop=True, inplace=True)

In [34]:
# Verificar cambios
directorio_full.head(50)

,name,descripcion,Latitude,Longitude,coordenadas,tipo,direccion,bd
0,DIF Estatal Baja California,DIF,32.663780,-115.464355,"32.6637802, -115.464355",GOB,"Av. Obregón 1290, Col. Nueva 2da. Secc., C.P. ...",Directorio OIM
1,Ángeles Sin Fronteras,Centros de Asistencia Social,32.662937,-115.494983,"32.66293697601438, -115.49498343917332",OSC / ONG,"Av. Reforma 244, Altos Zona Centro C.P.21100",Directorio OIM
2,Centro de Apoyo al Trabajor Migrante,Centros de Asistencia Social,32.660848,-115.479494,"32.660847720110326, -115.4794934911816",OSC / ONG,"Jabonera 6, Col. Nueva C.P. 21100",Directorio OIM
3,Centro de Apoyo al Trabajador Migrante: Comit...,Centros de Atención Social,32.660305,-115.459726,"32.6603047, -115.4597261",OSC / ONG,"Calle Jabonera #6, Colonia Nueva, Mexicali, Ba...",redes
4,Comedor Cobina A.C.,Centros de Asistencia Social,32.660301,-115.489875,"32.66030066068713, -115.48987485285113",OSC / ONG,"Avenida Lerdo 416, Col. Centro C.P. 21200",Directorio OIM
5,"Comedor Cobina, A.C.",Centros de Atención Social,32.660172,-115.492330,"32.6601716, -115.4923303",OSC / ONG,"Av. Lerdo 416, Col. Centro, C.P. 21200,Mexical...",redes
6,Juntos en el Camino,Centros de Asistencia Social,32.658954,-115.565696,"32.65895404522832, -115.56569597666147",OSC / ONG,"Garita 1, Carretera Mexicali-Calexico, Módulo 3",Directorio OIM
7,HIAS Mexicali,Centros de Atención Social,32.658493,-115.372497,"32.6584928, -115.3724971",OSC / ONG,"Calle México 151, Primera, 21100 Mexicali, B.C...",redes
8,Dispensario Médico Dr. Cañedo,Centros de Atención Social,32.658323,-115.437183,"32.6583229, -115.4371825",OSC / ONG,"Calle Sur # 701,Mexicali, Baja California",redes
9,Grupo de Ayuda para el Migrante de Mexicali A...,Centros de Atención Social,32.656253,-115.498342,"32.65625302822536, -115.49834185226172",OSC / ONG,"Av. Michoacan #1377, entre la 5ta y 6ta, Col. ...",redes


In [35]:
# Paso 1: Eliminar espacios al inicio en 'name'
directorio_full['name'] = directorio_full['name'].str.lstrip()

# Paso 2: Quitar acentos
def quitar_acentos(texto):
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    return texto

for columna in directorio_full.select_dtypes(include=['object']).columns:
    directorio_full[columna] = directorio_full[columna].apply(quitar_acentos)

In [36]:
directorio_full['bd'].unique()

array(['Directorio OIM', 'redes', 'ACNUR albergues', 'ACNUR servicios',
       'Oficinas de Atencion a Migrantes (OFAMs)', 'COMAR'], dtype=object)

### separamos en diferentes bds

In [27]:
Directorio_OIM = directorio_full[directorio_full['bd'] == 'Directorio OIM']


In [48]:
# Lista de valores a reemplazar
valores_a_reemplazar = ['Centros de Asistencia Social', 'Centro de Asistencia Social']

# Reemplazar los valores en la columna 'descripcion'
Directorio_OIM['descripcion'] = Directorio_OIM['descripcion'].replace(valores_a_reemplazar, 'Centros de Atencion Social')

# Verificar los valores únicos después del reemplazo
print(Directorio_OIM['descripcion'].unique())

['DIF' 'Centros de Atencion Social' 'Grupos Beta' 'Oficina Consular'
 'CNDH' 'INM' 'Oficinas de Atencion a Migrante' 'Salud' 'Mujeres'
 'FISCALIAS ESPECIALIZADAS PARA PERSONAS MIGRANTES' 'COMAR' 'ONU'
 'Consulados']


In [49]:
Directorio_OIM['descripcion'].unique()

array(['DIF', 'Centros de Atencion Social', 'Grupos Beta',
       'Oficina Consular', 'CNDH', 'INM',
       'Oficinas de Atencion a Migrante', 'Salud', 'Mujeres',
       'FISCALIAS ESPECIALIZADAS PARA PERSONAS MIGRANTES', 'COMAR', 'ONU',
       'Consulados'], dtype=object)

In [67]:
Directorio_OIM.shape

(1397, 8)

In [56]:
ACNUR_albergues = directorio_full[directorio_full['bd'] == 'ACNUR albergues']
ACNUR_servicios = directorio_full[directorio_full['bd'] == 'ACNUR servicios']

### directorios seleccionados:

In [53]:
CAS = Directorio_OIM[Directorio_OIM['descripcion'] == 'Centros de Atencion Social']

In [50]:
grupos_beta = Directorio_OIM[Directorio_OIM['descripcion'] == 'Grupos Beta']

In [52]:
salud = Directorio_OIM[Directorio_OIM['descripcion'] == 'Salud']

In [68]:
import pandas as pd

# Concatenar DataFrames
resultante_df = pd.concat([grupos_beta, salud], ignore_index=True)


In [69]:
# Guardar el DataFrame resultante en un archivo .xlsx
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/cruzrojagruposbeta.xlsx'  # Cambia esto por tu ruta y nombre de archivo real

resultante_df.to_excel(ruta_archivo, index=False, engine='openpyxl')


# dataframe completo

Incluye centros de atención social extraidos de los directorios de OIM, Redes, ACNUR.

In [70]:
df1 = pd.read_excel('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/instituciones_sc.xlsx')

In [73]:
grid = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv')


In [76]:
from shapely.wkt import loads
if 'geometry' in grid.columns:
    # Convertir cadenas de geometría a objetos geométricos usando `loads` de Shapely
    grid['geometry'] = grid['geometry'].apply(lambda x: loads(x) if isinstance(x, str) else x)

    # Ahora que todos los valores en 'geometry' son objetos geométricos, crea el GeoDataFrame
    grid = gpd.GeoDataFrame(grid, geometry='geometry')
else:
    print("La columna 'geometry' no existe en 'grid'.")

In [72]:
# Paso 1: Eliminar espacios al inicio en 'name'
df1['name'] = df1['name'].str.lstrip()

# Paso 2: Quitar acentos
def quitar_acentos(texto):
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    return texto

for columna in directorio_full.select_dtypes(include=['object']).columns:
    df1[columna] = df1[columna].apply(quitar_acentos)

In [79]:
# Convertir df1 en GeoDataFrame
gdf1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.Longitude, df1.Latitude))

# Aquí estoy asumiendo que tus datos están en WGS84 (EPSG:4326)
gdf1.set_crs(epsg=4326, inplace=True)
grid.set_crs(epsg=4326, inplace=True)

# Realizar la operación sjoin
gdf1_sjoined = gpd.sjoin(gdf1, grid, how="left", op="within")

# Asignar los 'ID_Poligono' a df1 desde el resultado del sjoin
df1["ID_Poligono"] = gdf1_sjoined["ID_Poligono"]

# Verificar los resultados
print(df1.head())

                                                name  \
0                              Angeles Sin Fronteras   
1               Centro de Apoyo al Trabajor Migrante   
2  Centro de Apoyo al Trabajador Migrante: Comite...   
3                                Comedor Cobina A.C.   
4                              Comedor Cobina, A.C.    

                    descripcion   Latitude   Longitude  \
0  Centros de Asistencia Social  32.662937 -115.494983   
1  Centros de Asistencia Social  32.660848 -115.479494   
2  Centros de Asistencia Social  32.660305 -115.459726   
3  Centros de Asistencia Social  32.660301 -115.489875   
4  Centros de Asistencia Social  32.660172 -115.492330   

                              coordenadas       tipo  \
0  32.66293697601438, -115.49498343917332  OSC / ONG   
1  32.660847720110326, -115.4794934911816  OSC / ONG   
2                32.6603047, -115.4597261  OSC / ONG   
3  32.66030066068713, -115.48987485285113  OSC / ONG   
4                32.6601716, -115.

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [81]:
ruta_archivo12 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Cargar el archivo .xls en un DataFrame
df_ciudades = pd.read_csv(ruta_archivo12)

In [82]:
# Primero, nos aseguramos de que df_ciudades tenga valores únicos de 'ID_Poligono'
df_ciudades_unicos = df_ciudades.drop_duplicates(subset='ID_Poligono', keep='first')

# Ahora realizamos el merge, esta vez debería evitar la duplicación de filas en df1
df_merged = df1.merge(df_ciudades_unicos[['ID_Poligono', 'estado']], on='ID_Poligono', how='left')

# Renombramos las columnas si es necesario y eliminamos las adicionales que no necesitamos
# Este paso depende de si 'merge' crea nuevas columnas o no
df_merged.rename(columns={'estado_y': 'estado'}, inplace=True)
if 'estado_x' in df_merged.columns:
    df_merged.drop('estado_x', axis=1, inplace=True)

# Actualizamos df1 con los datos fusionados
df1 = df_merged

# Verificamos los resultados
print(df1[['ID_Poligono', 'estado']].head())


  ID_Poligono           estado
0       PG132  Baja California
1       PG132  Baja California
2       PG132  Baja California
3       PG132  Baja California
4       PG132  Baja California


In [84]:
# Asumiendo que df1 y df_ciudades ya están cargados

# Primero, nos aseguramos de que df_ciudades tenga valores únicos de 'ID_Poligono'
df_ciudades_unicos = df_ciudades.drop_duplicates(subset='ID_Poligono', keep='first')

# Ahora realizamos el merge, esta vez debería evitar la duplicación de filas en df1
df_merged = df1.merge(df_ciudades_unicos[['ID_Poligono', 'ciudad']], on='ID_Poligono', how='left')

# Renombramos las columnas si es necesario y eliminamos las adicionales que no necesitamos
# Este paso depende de si 'merge' crea nuevas columnas o no
df_merged.rename(columns={'ciudad_y': 'ciudad'}, inplace=True)
if 'estado_x' in df_merged.columns:
    df_merged.drop('ciudad_x', axis=1, inplace=True)

# Actualizamos df1 con los datos fusionados
df1 = df_merged

# Verificamos los resultados
print(df1[['ID_Poligono', 'ciudad']].head())


  ID_Poligono                     ciudad
0       PG132  Mexicali, Baja California
1       PG132  Mexicali, Baja California
2       PG132  Mexicali, Baja California
3       PG132  Mexicali, Baja California
4       PG132  Mexicali, Baja California


In [91]:
df1.head(60)

,name,ciudad,estado,tipo,subtipo,latitud,longitud,ID_Poligono,direccion
0,Angeles Sin Fronteras,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.662937,-115.494983,PG132,"Av. Reforma 244, Altos Zona Centro C.P.21100"
1,Centro de Apoyo al Trabajor Migrante,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660848,-115.479494,PG132,"Jabonera 6, Col. Nueva C.P. 21100"
2,Centro de Apoyo al Trabajador Migrante: Comite...,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660305,-115.459726,PG132,"Calle Jabonera #6, Colonia Nueva, Mexicali, Ba..."
3,Comedor Cobina A.C.,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660301,-115.489875,PG132,"Avenida Lerdo 416, Col. Centro C.P. 21200"
4,"Comedor Cobina, A.C.","Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660172,-115.492330,PG132,"Av. Lerdo 416, Col. Centro, C.P. 21200,Mexical..."
5,Juntos en el Camino,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.658954,-115.565696,PG132,"Garita 1, Carretera Mexicali-Calexico, Modulo 3"
6,HIAS Mexicali,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.658493,-115.372497,PG132,"Calle Mexico 151, Primera, 21100 Mexicali, B.C..."
7,Dispensario Medico Dr. Canedo,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.658323,-115.437183,PG132,"Calle Sur # 701,Mexicali, Baja California"
8,Grupo de Ayuda para el Migrante de Mexicali A.C.,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.656253,-115.498342,PG132,"Av. Michoacan #1377, entre la 5ta y 6ta, Col. ..."
9,"Grupo de Ayuda para el Migrante de Mexicali, A.C.","Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.655935,-115.498796,PG132,"Michoacan 1377, Col. Pueblo Nuevo C.P. 21120"


In [88]:
df1 = df1.rename(columns={
    'descripcion': 'subtipo',
    'Latitude': 'latitud',
    'Longitude': 'longitud'
})


In [89]:
df1 = df1.drop(columns=['coordenadas'])


In [90]:
df1 = df1[['name', 'ciudad', 'estado', 'tipo', 'subtipo', 'latitud', 'longitud', 'ID_Poligono', 'direccion']]


In [94]:
# Guardar el DataFrame resultante en un archivo .xlsx
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/instituciones_sc_1.csv'  # Cambia esto por tu ruta y nombre de archivo real

df1.to_csv(ruta_archivo, index=False)

### mujeres, niños y familias

In [39]:
DIF = directorio_full[directorio_full['descripcion'] == 'DIF']

In [40]:
DIF

,name,descripcion,Latitude,Longitude,coordenadas,tipo,direccion,bd
0,DIF Estatal Baja California,DIF,32.663780,-115.464355,"32.6637802, -115.464355",GOB,"Av. Obregon 1290, Col. Nueva 2da. Secc., C.P. ...",Directorio OIM
209,DIF Estatal Chihuahua,DIF,28.673408,-106.065943,"28.6734077, -106.065943",GOB,"Av. Tecnologico 2903, Col. Magisterial C.P. 31...",Directorio OIM
333,DIF Estatal Nuevo Leon,DIF,25.671604,-100.354969,"25.671604, -100.3549688",GOB,"Dr. Ignacio Morones Prieto 600, Col. Independe...",Directorio OIM
441,DIF Estatal Coahuila,DIF,25.370785,-100.999385,"25.3707847, -100.9993851",GOB,"Paseo de las Arboledas s/n, Col. Chapultepec, ...",Directorio OIM
445,DIF Estatal Sinaloa,DIF,24.794813,-107.450420,"24.794812894548972, -107.45042009778955",GOB,"Blvd. Miguel Tamayo 3000 Norte, Col. Desarroll...",Directorio OIM
452,DIF Estatal Durango,DIF,24.045363,-104.654695,"24.045363404030446, -104.65469518488028",GOB,"Blvd. Jose Maria Patoni, Mnz. 105, esq. Calle ...",Directorio OIM
462,DIF Estatal Tamaulipas,DIF,23.719263,-99.165535,"23.719263449263618, -99.1655348229578",GOB,"Calz. Gral. Luis Caballero 297, Col. Centro, C...",Directorio OIM
481,DIF Estatal Baja California Sur,DIF,22.904202,-109.887892,"22.90420185, -109.88789179405553",GOB,"Carretera Transpeninsular al Norte Km. 4.5, Co...",Directorio OIM
485,DIF Estatal Zacatecas,DIF,22.756046,-102.589990,"22.7560462, -102.5899901",GOB,"Paseo la Encantada 102, Col. La Encantada, C.P...",Directorio OIM
495,DIF Estatal Sonora,DIF,22.159465,-100.927149,"22.1594649, -100.9271488",GOB,"Blvd. Luis Encinas s/n, Col. San Benito, C.P. ...",Directorio OIM
